<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/PyTorch/Basic/ProtAlbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtAlbert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 778kB 4.7MB/s 
     |████████████████████████████████| 3.0MB 20.2MB/s 
     |████████████████████████████████| 890kB 40.8MB/s 
     |████████████████████████████████| 1.1MB 54.3MB/s 


In [2]:
import torch
from transformers import AutoTokenizer, AutoModel, pipeline
import re
import numpy as np
import os
import requests
from tqdm.auto import tqdm

<b>2. Load the vocabulary and ProtAlbert Model<b>

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_albert", do_lower_case=False )

In [4]:
model = AutoModel.from_pretrained("Rostlab/prot_albert")

<b>3. Load the model into the GPU if avilabile and switch to inference mode<b>

In [5]:
fe = pipeline('feature-extraction', model=model, tokenizer=tokenizer,device=0)

<b>4. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [6]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [7]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>5. Extracting sequences' features and covert the output to numpy if needed<b>

In [8]:
embedding = fe(sequences_Example)

In [9]:
embedding = np.array(embedding)

In [10]:
print(embedding)

[[[ 0.05823839  0.0166735  -0.07323111 ...  0.05722338  0.05712228
   -0.01743178]
  [ 0.05041128  0.01670685 -0.06041763 ...  0.03879201  0.0674293
   -0.02243809]
  [ 0.04481388  0.02007382 -0.06041713 ...  0.01449793  0.07357242
   -0.02993674]
  ...
  [ 0.03966058  0.04050258 -0.01892234 ...  0.02171973  0.06361844
   -0.04444959]
  [ 0.04467377  0.02889517 -0.06304097 ...  0.00830418  0.0829025
   -0.02485315]
  [ 0.05313198  0.01662794 -0.07059952 ...  0.04868674  0.06335221
   -0.01911487]]

 [[-0.03029662  0.01223785 -0.02814536 ... -0.12239366 -0.11794586
    0.06286722]
  [-0.03166402  0.01354338 -0.02900368 ... -0.11964945 -0.11535365
    0.07926654]
  [-0.04238719  0.01891202 -0.02534003 ... -0.12101925 -0.10938749
    0.09651614]
  ...
  [-0.03416557  0.01334485 -0.02930344 ... -0.12059727 -0.1147826
    0.07598283]
  [-0.04335115  0.02185646 -0.02340434 ... -0.12275542 -0.10738459
    0.1049581 ]
  [-0.04291803  0.02312845 -0.02060059 ... -0.12334678 -0.10542988
    0.108

<b>Optional: Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by ProtAlbert model<b>

In [11]:
features = [] 

for seq_num in range(len(embedding)):
    seq_len = len(sequences_Example[seq_num].replace(" ", ""))
    start_Idx = 1
    end_Idx = seq_len+1
    seq_emd = embedding[seq_num][start_Idx:end_Idx]
    features.append(seq_emd)

In [12]:
print(features)

[array([[ 0.05041128,  0.01670685, -0.06041763, ...,  0.03879201,
         0.0674293 , -0.02243809],
       [ 0.04481388,  0.02007382, -0.06041713, ...,  0.01449793,
         0.07357242, -0.02993674],
       [ 0.04433505,  0.02113645, -0.0598816 , ...,  0.01485567,
         0.07245695, -0.02932317],
       ...,
       [ 0.04479406,  0.01773301, -0.06211168, ...,  0.01833694,
         0.0708717 , -0.02901287],
       [ 0.03966058,  0.04050258, -0.01892234, ...,  0.02171973,
         0.06361844, -0.04444959],
       [ 0.04467377,  0.02889517, -0.06304097, ...,  0.00830418,
         0.0829025 , -0.02485315]]), array([[-0.03166402,  0.01354338, -0.02900368, ..., -0.11964945,
        -0.11535365,  0.07926654],
       [-0.04238719,  0.01891202, -0.02534003, ..., -0.12101925,
        -0.10938749,  0.09651614],
       [-0.04247086,  0.02128087, -0.02400087, ..., -0.12410779,
        -0.10866657,  0.10307116],
       [-0.04353952,  0.02157599, -0.02257544, ..., -0.12234627,
        -0.10860701,